# A Super Toy Regression Example

To use GPU, model and train test data need to be loaded to GPU. cuda is needed as well to communicate with GPU. Other parts remain the same as trainig on CPU.

In [1]:
import numpy as np
import torch
import torch.nn as nn

## Prepare data
This time we just use simple mock data and don't do train test split.

In [2]:
x_values = [i for i in range(10)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1, 1)
x_train.shape

(10, 1)

In [3]:
y_values = [2*i + 1 for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)
y_train.shape

(10, 1)

## Build model

In [4]:
class LinearRegressionModel(nn.Module):
  def __init__(self, input_dim, output_dim):
    super(LinearRegressionModel, self).__init__()
    self.linear = nn.Linear(input_dim, output_dim)

  def forward(self, x):
    out = self.linear(x)
    return out

In [5]:
input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)

# The following two lines is for GPU
# device = torch.device("cuda: 0" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [6]:
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

## Set hyperparameters and loss function

In [7]:
epochs = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [8]:
for epoch in range(epochs):

  # convert np array to torch
  inputs = torch.from_numpy(x_train)
  labels = torch.from_numpy(y_train)

  # the following is GPU version
  # inputs = torch.from_numpy(x_train).to(device)
  # labels = torch.from_numpy(y_train).to(device)

  # clear gradient, otherwise it accumulates
  optimizer.zero_grad()

  # forward propagation
  # don't even need to call the forward method!
  outputs = model(inputs)

  # compute loss
  loss = criterion(outputs, labels)

  # backward propagation
  loss.backward()

  # update parameters
  optimizer.step()

  # print results
  if epoch % 50 ==0:
    print(f"epoch: {epoch}, loss: {loss.item()}")

epoch: 0, loss: 256.68243408203125
epoch: 50, loss: 0.005956332199275494
epoch: 100, loss: 0.0033808674197643995
epoch: 150, loss: 0.001919012749567628
epoch: 200, loss: 0.0010892294812947512
epoch: 250, loss: 0.0006182418437674642
epoch: 300, loss: 0.00035092068719677627
epoch: 350, loss: 0.00019919045735150576
epoch: 400, loss: 0.00011306182568660006
epoch: 450, loss: 6.417484837584198e-05
epoch: 500, loss: 3.642580486484803e-05
epoch: 550, loss: 2.067586683551781e-05
epoch: 600, loss: 1.1736109627236146e-05
epoch: 650, loss: 6.662109626631718e-06
epoch: 700, loss: 3.7825798244739417e-06
epoch: 750, loss: 2.1465443751367275e-06
epoch: 800, loss: 1.218227907884284e-06
epoch: 850, loss: 6.917414907547936e-07
epoch: 900, loss: 3.9262766904357704e-07
epoch: 950, loss: 2.2278811684373068e-07


## Use the trained model to predict results

In [9]:
predicted = model(torch.from_numpy(x_train))
# convert to numpy, for later usage like matplotlib etc
predicted = predicted.data.numpy()
predicted

array([[ 1.0006608],
       [ 3.0005555],
       [ 5.00045  ],
       [ 7.0003448],
       [ 9.000239 ],
       [11.000134 ],
       [13.000029 ],
       [14.999923 ],
       [16.999817 ],
       [18.999712 ]], dtype=float32)

## Save model

In [10]:
# saved as dictionary
# save to where??
torch.save(model.state_dict(), 'model.pkl')

## Load model

In [11]:
model.load_state_dict(torch.load('model.pkl'))

<All keys matched successfully>